# Data Loading

In [2]:
import pandas as pd
                                                                                                                                             
#load the data
seg=pd.read_csv('./file.csv',encoding='latin1')

#categorical columns
categorical = seg.select_dtypes(include='object')

#numerical columns
numerical = seg._get_numeric_data()

# concat the numerical variable and categorical columns
df = pd.concat([seg[numerical.columns.tolist()],pd.get_dummies(seg[categorical.columns.tolist()])],axis=1)
df.head()

,customer id,Nearest_distance_cosco,Net Sales,Age,children count,Martial status,Loyality_flag,coupon amount,Income,Orders groceries,...,Brands_ Shiseido Co,Brands_ Spectrum Brands Holdings,Brands_ Stanley Black & Decker,Brands_ Svenska Cellulosa AB SCA,Brands_ Swatch Group SA*,Brands_ Tyson Foods,Brands_ Unilever N,Brands_ VF Corp,Brands_ WH Group Ltd,Brands_ Whirlpool Corp
0,5102,4,367941,58,4,0,0,6,4628,6,...,0,0,0,0,0,0,0,0,0,0
1,3549,33,171305,98,1,0,0,43,4074,7,...,0,0,0,0,0,0,0,0,0,0
2,5885,28,173439,18,2,0,0,42,4910,5,...,0,0,0,0,0,0,0,0,0,0
3,7381,40,997988,87,3,0,1,38,3297,5,...,0,0,0,0,0,0,0,0,0,0
4,2713,5,898554,31,2,0,0,24,4940,4,...,0,0,0,0,0,0,0,0,0,0


# Split the dataset and apply the Model

In [3]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

X = df.drop(['customer id','segments'],axis=1)
y = df['segments']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42,shuffle=True)
classifier = xgb.XGBClassifier(random_state=2)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
f1 = f1_score(y_test,y_pred,average='macro')
print("F1-scpore:",f1)
xgb_cr = classification_report(y_test,y_pred)
print("Classification Report:/n",xgb_cr)

F1-scpore: 0.1748557401852023
Classification Report:/n               precision    recall  f1-score   support

           1       0.15      0.17      0.16       247
           2       0.23      0.16      0.19       288
           3       0.19      0.24      0.21       241
           4       0.13      0.14      0.14       231
           5       0.18      0.14      0.16       251
           6       0.18      0.21      0.20       242

    accuracy                           0.18      1500
   macro avg       0.18      0.18      0.17      1500
weighted avg       0.18      0.18      0.18      1500



# Predictor Check

In [5]:
from sklearn.model_selection import GridSearchCV

parameters={'learning_rate':[0.1,0.15,0.2,0.25,0.3],
            'max_depth':range(1,3)}

# instantiate the GridsearchCV 
grid_search = GridSearchCV(estimator = classifier,param_grid = parameters,n_jobs = -1,verbose=4)

# fit the model
grid_search = grid_search.fit(X_train, y_train)

# predict the result
grid_predictions =grid_search.predict(X_test)

#calculate the f1 score
grid_f1= f1_score(y_test, grid_predictions, average='macro')
print(grid_f1)

# calssification report
report=classification_report(y_test, grid_predictions)
print(report)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  48 out of  50 | elapsed: 10.0min remaining:   24.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 10.2min finished


0.1773890153382315
              precision    recall  f1-score   support

           1       0.19      0.21      0.20       247
           2       0.20      0.12      0.15       288
           3       0.18      0.29      0.22       241
           4       0.14      0.13      0.14       231
           5       0.14      0.11      0.12       251
           6       0.22      0.24      0.23       242

    accuracy                           0.18      1500
   macro avg       0.18      0.18      0.18      1500
weighted avg       0.18      0.18      0.18      1500



# Predictor check using ensemble method

In [6]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=2)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

f1 = f1_score(y_test,y_pred,average='macro')
print(f1)
report = classification_report(y_test,y_pred)
print(report)

0.16116249335524743
              precision    recall  f1-score   support

           1       0.18      0.21      0.19       247
           2       0.16      0.10      0.13       288
           3       0.15      0.22      0.18       241
           4       0.16      0.17      0.17       231
           5       0.18      0.14      0.16       251
           6       0.15      0.14      0.14       242

    accuracy                           0.16      1500
   macro avg       0.16      0.16      0.16      1500
weighted avg       0.16      0.16      0.16      1500

